In [1]:
#!fsharp
#r "nuget:Microsoft.ML,1.4.0"
#r "nuget:Microsoft.ML.FastTree,1.4.0"
#r "nuget:XPlot.Plotly,3.0.1"

In [1]:
#!fsharp
open System
open System.IO
open Microsoft.ML
open Microsoft.ML.Data

In [1]:
#!fsharp
/// Type representing the text to run sentiment analysis on.
[<CLIMutable>] 
type SentimentIssue = 
    { 
        [<LoadColumn(0)>]
        Label : bool

        [<LoadColumn(2)>]
        Text : string 
    }

/// Result of sentiment prediction.
[<CLIMutable>]
type  SentimentPrediction = 
    { 
        // ColumnName attribute is used to change the column name from
        // its default value, which is the name of the field.
        [<ColumnName("PredictedLabel")>]
        Prediction : bool; 

        // No need to specify ColumnName attribute, because the field
        // name "Probability" is the column name we want.
        Probability : float32; 

        Score : float32 
    }

In [1]:
#!fsharp
let ctx = MLContext()

In [1]:
#!fsharp
// Load in the data
let dataView = ctx.Data.LoadFromTextFile<SentimentIssue>(Path.Combine("data", "wikiDetoxAnnotated40kRows.tsv"), hasHeader = true)

// Split data into test and training sets
let trainTestSplit = ctx.Data.TrainTestSplit(dataView, testFraction = 0.2)
let trainingDataView = trainTestSplit.TrainSet
let testDataView = trainTestSplit.TestSet

In [1]:
#!fsharp
let pipeline = ctx.Transforms.Text.FeaturizeText("Features", "Text")

In [1]:
#!fsharp
let trainer = ctx.BinaryClassification.Trainers.FastTree(labelColumnName = "Label", 
                                                         featureColumnName = "Features")
let trainingPipeline = pipeline.Append(trainer)

In [1]:
#!fsharp
let trainedModel = pipeline.Fit(trainingDataView)

In [1]:
#!fsharp
printfn "===== Evaluating Model's accuracy with Test data ====="
let predict = trainedModel.Transform testDataView
let metrics = ctx.BinaryClassification.Evaluate(predict, "Label", "Score")